
# Bokeh for Time Series Analysis
<hr style="border: 2px solid black;">


<img src="./images/bokeh.png" alt="bokeh Logo" width="1000"/>
<hr style="border: 2px solid black;">

<img src="./images/bokeh_at_ag_glance.png" alt="bokeh Logo" width="1000"/>
<hr style="border: 2px solid black;">
**Introduction to Bokeh**
Bokeh is an interactive visualization library for Python that targets modern web browsers for presentation.
Unlike Matplotlib, which is primarily designed for static plots, Bokeh excels at creating
interactive plots and dashboards. It can handle large datasets and streaming data,
making it suitable for real-time applications.

**Key Features of Bokeh:**

* **Interactivity:** Built-in support for zooming, panning, hovering, and other interactive tools.
* **Web-Focused:** Generates HTML and JavaScript, making it easy to embed plots in web pages.
* **High Performance:** Can handle large datasets efficiently.
* **Versatility:** Supports a wide range of plot types (lines, bars, scatter plots, etc.).

<hr style="border: 2px solid black;">


**Documentation:**

For comprehensive documentation, please refer to the official Bokeh website: [https://docs.bokeh.org/en/latest/](https://docs.bokeh.org/en/latest/)


<hr style="border: 2px solid black;">


**Lab Exercise:**

Your task is to recreate the time series analysis lab we previously conducted using Pandas,
Matplotlib, and Seaborn, but this time, utilize the Bokeh library for visualization.
This will involve:

1.  Loading and preprocessing the "AirPassengersDates.csv" dataset.
2.  Creating interactive Bokeh plots for:
    * Time series line plots
    * Bar plots of aggregated data
    * Visualizing mean and standard deviation
    * Outlier detection
    * Resampling (upsampling and downsampling)
    * Lag analysis
    * Autocorrelation

Pay close attention to Bokeh's features for interactivity (tools, hover effects) and
its handling of data sources. Aim to replicate the insights and visualizations
from the previous lab while leveraging Bokeh's strengths.

Good luck!
<hr style="border: 2px solid black;">

In [16]:
!pip install bokeh



In [17]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10
from bokeh.io import output_notebook

output_notebook()

output_notebook()  # Affiche les figures Bokeh directement dans Jupyter


Loading BokehJS ...

Loading BokehJS ...

In [18]:
source = ColumnDataSource(data={
    'x': df.index,
    'y': df['#Passengers']
})

p = figure(x_axis_type='datetime', title='Monthly Air Passengers Over Time',
           width=800, height=300, tools="pan,box_zoom,reset,wheel_zoom")

hover = HoverTool(
    tooltips=[
        ('Date', '@x{%F}'),
        ('Passengers', '@y')
    ],
    formatters={'@x': 'datetime'},
    mode='vline'
)
p.add_tools(hover)

p.line('x', 'y', source=source, line_width=2, color='navy', legend_label='Passengers')
p.circle('x', 'y', source=source, size=4, color='orange')

p.legend.location = "top_left"
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Number of Passengers'

show(p)


In [23]:
# Cellule 3 — Bar plot: total annuel des passagers

# On s'assure que 'Date' est bien en datetime et en index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Regroupement annuel
df_yearly = df.resample('Y').sum()

# Préparation de l'index pour Bokeh
df_yearly.index = df_yearly.index.year.astype(str)  # Années en string

# Source pour Bokeh
source_year = ColumnDataSource(data={
    'year': list(df_yearly.index),
    'passengers': df_yearly['#Passengers']
})

# Création du graphe
p2 = figure(x_range=list(df_yearly.index), 
            title="Total Passengers per Year",
            height=300, width=800, tools="hover")

p2.vbar(x='year', top='passengers', width=0.6, source=source_year, color=Category10[10][0])

# Axes
p2.xaxis.axis_label = 'Year'
p2.yaxis.axis_label = 'Total Passengers'

# Tooltip
p2.add_tools(HoverTool(
    tooltips=[("Year", "@year"), ("Passengers", "@passengers")]
))

show(p2)


C:\Users\enele\AppData\Local\Temp\ipykernel_33760\950402111.py:8: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  df_yearly = df.resample('Y').sum()


In [24]:
rolling_mean = df['#Passengers'].rolling(window=12).mean()
rolling_std = df['#Passengers'].rolling(window=12).std()

source_stats = ColumnDataSource(data={
    'x': df.index,
    'y': df['#Passengers'],
    'mean': rolling_mean,
    'std_up': rolling_mean + rolling_std,
    'std_down': rolling_mean - rolling_std
})

p3 = figure(x_axis_type='datetime', title='Mean and Standard Deviation',
            width=800, height=300, tools="hover,pan,box_zoom,reset")

p3.line('x', 'y', source=source_stats, legend_label='Passengers', line_color='gray')
p3.line('x', 'mean', source=source_stats, legend_label='Mean', line_color='green')
p3.line('x', 'std_up', source=source_stats, legend_label='+1 Std Dev', line_color='blue', line_dash='dashed')
p3.line('x', 'std_down', source=source_stats, legend_label='-1 Std Dev', line_color='red', line_dash='dashed')

p3.legend.location = 'top_left'

show(p3)
